In [1]:
import sys
sys.path.append("./Preprocessing")

import pandas as pd
from feature_engineering import FeatureEngineering
from Resampling import Resampling

In [2]:
dataset = pd.read_csv('C:/Users/ruben/OneDrive/Desktop/Datasets/original_dataset.csv')

# Apply feature engineering on the dataset
fe = FeatureEngineering(dataset)
processed_dataset = fe.apply_feature_engineering()

# Apply resampling on the dataset
resampler = Resampling(processed_dataset, test_size=0.4, random_state=42)
final_dataset = resampler.apply_resampling()

AttributeError: 'Resampling' object has no attribute 'process'